In [ ]:

import os
from langchain.chat_models import init_chat_model

from typing import Annotated
import csv
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition



class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = init_chat_model("openai:gpt-4.1")

@tool
def get_deposits(customer_id: str = None) -> list:
    """Returns all bank deposits as a list of dicts. Optionally filter by customer_id."""
    print("get_deposits tool called with:", customer_id)
    with open("src/assets/Deposits.csv", newline="", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        deposits = list(reader)
    if customer_id:
        deposits = [d for d in deposits if d.get("CustomerID") == customer_id]
    return deposits

tools = [get_deposits]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [18]:
graph.invoke({"messages" : {"role" : "user", "content":"Return all bank deposits."}})

get_deposits tool called with: None


{'messages': [HumanMessage(content='Return all bank deposits.', additional_kwargs={}, response_metadata={}, id='2cebf99a-534f-4306-9840-93317f4f223c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0SxJK8oZYrNOzVbJFvGAz98z', 'function': {'arguments': '{}', 'name': 'get_deposits'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 69, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-BnWwMtPaIWjG0Y8EwUAwnC608Ddmz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a647cec9-0984-43ed-83e6-d5c771d24904-0', tool_calls=[{'name': 'get_deposits', 'args': {}, 'id': 'call_0SxJK8oZYrNOzVbJFvGAz98z', 'type': 

In [ ]:

import psycopg2
import os

conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME", "sarbotdb"),
    user=os.getenv("DB_USER", "sarbotdb_owner"),
    password=os.getenv("NEON_PASS"),
    host=os.getenv("DB_HOST", "ep-shy-cloud-a8eg2rzb-pooler.eastus2.azure.neon.tech"),
    port=os.getenv("DB_PORT", "5432"),
    sslmode="require",
)
query = "SELECT * FROM public.transactions"
cursor = conn.cursor()
cursor.execute(query)
deposits = cursor.fetchall()
cursor.close()
conn.close()


In [8]:
deposits

[(1, 5, 23, datetime.date(2025, 6, 24), Decimal('9000.00'), 'credit'),
 (2, 5, 22, datetime.date(2025, 6, 23), Decimal('9900.00'), 'credit'),
 (3, 5, 22, datetime.date(2025, 6, 22), Decimal('9980.00'), 'credit'),
 (4, 8, 23, datetime.date(2025, 6, 26), Decimal('155.00'), 'debit'),
 (5, 9, 44, datetime.date(2025, 6, 25), Decimal('457.00'), 'credit'),
 (6, 1, 123, datetime.date(2025, 6, 28), Decimal('130800.00'), 'debit'),
 (7, 1, 123, datetime.date(2025, 6, 27), Decimal('130800.00'), 'credit')]